In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler, DPMSolverMultistepScheduler, DDIMScheduler
from PIL import Image
import os

# Install required packages
!pip install diffusers==0.31.0 transformers accelerate safetensors --upgrade
# If you need to authenticate to HuggingFace:
# from huggingface_hub import login
# login()  # Insert your token if the model requires authentication

print("Libraries installed and imported successfully.")

In [ ]:
# Load the model and set the tokenizer limits
pipe = StableDiffusionXLPipeline.from_pretrained(
    "Laxhar/noobai-XL-1.1", 
    torch_dtype=torch.float16, 
    scheduler=EulerDiscreteScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler"), 
    use_safetensors=True  # or False, depending on the repository
).to("cuda")

# Remove token length limits
if pipe.tokenizer is not None:
    pipe.tokenizer.model_max_length = 100000

if hasattr(pipe, "tokenizer_2") and pipe.tokenizer_2 is not None:
    pipe.tokenizer_2.model_max_length = 100000

print("Pipeline loaded and tokenizer limits set.")

In [ ]:
# Function to set the diffusion sampler
def set_diffusion_sampler(pipeline, sampler_name="euler"):
    sampler_name = sampler_name.lower().strip()
    if sampler_name == "euler":
        scheduler = EulerDiscreteScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler")
    elif sampler_name == "ddim":
        scheduler = DDIMScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler")
    elif sampler_name == "dpm":
        scheduler = DPMSolverMultistepScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler")
    else:
        print(f"Unknown sampler: {sampler_name}. Using EulerDiscreteScheduler by default.")
        scheduler = EulerDiscreteScheduler.from_pretrained("Laxhar/noobai-XL-1.1", subfolder="scheduler")

    pipeline.scheduler = scheduler
    print(f"Sampler set to: {sampler_name}")

# Example call:
set_diffusion_sampler(pipe, "ddim")

In [ ]:
# Function to generate images with different parameters
def generate_image(
    pipeline,
    prompt,
    negative_prompt="",
    num_inference_steps=50,
    guidance_scale=7.5,
    width=768,
    height=768,
    seed=42
):
    generator = torch.manual_seed(seed)

    image = pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        width=width,
        height=height,
        generator=generator
    ).images[0]

    return image

print("Image generation function defined.")

In [ ]:
# Placeholder function for hires-fix / 8x NKMD-Superscale
def hires_fix(image, scale_factor=2):
    # Placeholder for hires-fix implementation
    # Example if using an external library such as real_esrgan or BSRGAN:
    # upsampler = RealESRGANer(
    #     model_path='weights/RealESRGAN_x4plus.pth',
    #     scale=4,
    #     etc...
    # )
    # upscaled_image = upsampler.enhance(image, outscale=scale_factor)[0]

    # Return the original image for now
    return image

print("Hires-fix function defined.")

In [ ]:
# Example usage of the image generation function
example_prompt = "A beautiful futuristic cityscape at sunset, ultra-detailed"
negative_prompt = "blurry, low quality"
steps = 30
cfg = 7.0
img_width = 512
img_height = 512
my_seed = 123

result_image = generate_image(
    pipeline=pipe,
    prompt=example_prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=steps,
    guidance_scale=cfg,
    width=img_width,
    height=img_height,
    seed=my_seed
)
result_image.show()

# Optional: Call hires-fix
# result_image_upscaled = hires_fix(result_image, scale_factor=8)
# result_image_upscaled.show()

print("Image generation completed!")